# Проект по SQL

## Описание проекта.

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Цель исследования – провести анализ данных сервиса книг и решить следующие задачи:
- сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры;
- определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных**

Таблица books:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

## Доступ к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Исследование данных

Прежде чем выполнить поставленные задачи, посмотрим на данные в таблицах.

In [3]:
# таблица books
query = '''SELECT * FROM books  LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# посмотрим на количество строк
query = '''SELECT COUNT (*) FROM books'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,1000


Таблица books содержит 1000 строк и следующие столбцы: book_id, author_id, title, num_pages, publication_date, publisher_id.

In [5]:
# таблица authors 
query = '''SELECT * FROM authors  LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
# посмотрим на количество строк
query = '''SELECT COUNT (*) FROM authors'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,636


Таблица authors содержит 636 строк и следующие столбцы: author_id, author.

In [7]:
# таблица publishers 
query = '''SELECT * FROM publishers  LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
# посмотрим на количество строк
query = '''SELECT COUNT (*) FROM publishers'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,340


Таблица publishers содержит 340 строк и следующие столбцы: publisher_id, publisher.

In [9]:
# таблица ratings 
query = '''SELECT * FROM ratings  LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
# посмотрим на количество строк
query = '''SELECT COUNT (*) FROM ratings'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,6456


Таблица ratings содержит 6456 строк и следующие столбцы: rating_id, book_id, username, rating.

In [11]:
# таблица reviews 
query = '''SELECT * FROM reviews  LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
# посмотрим на количество строк
query = '''SELECT COUNT (*) FROM reviews'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


Таблица reviews содержит 2793 строки и следующие столбцы: review_id, book_id, username, text.

Таким образом, были выгруженны все таблицы и есть представление какие данные в них. Теперь можно приступать к выполнению задач.

## Решение задач

**1. Посчитаем, сколько книг вышло после 1 января 2000 года.**

In [13]:
query1= ''' 
SELECT COUNT(*) 
FROM books
WHERE publication_date > '2000-1-1'; 
''' 
con=engine.connect()
pd.io.sql.read_sql(sql=text(query1), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг.

**2. Для каждой книги посчитаем количество обзоров и среднюю оценку.**

In [14]:
query2 = '''SELECT b.book_id,
            title,
            COUNT(DISTINCT rev.review_id) AS count_review, AVG(r.rating) AS rating_mean
            FROM books AS b LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
            LEFT JOIN ratings AS r ON rev.book_id = r.book_id
            GROUP BY b.book_id
            ORDER BY count_review DESC, rating_mean DESC;
           
         ''' 
con=engine.connect()
pd.io.sql.read_sql(sql=text(query2), con = con)

,book_id,title,count_review,rating_mean
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,387,Leonardo's Notebooks,0,NaN
996,221,Essential Tales and Poems,0,NaN
997,191,Disney's Beauty and the Beast (A Little Golden...,0,NaN
998,808,The Natural Way to Draw,0,NaN


 Не все книги оценены, есть книги без оценок. Больше всего оценок у Twilight (Twilight #1).	

**3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры.**

In [15]:
query3 = '''SELECT p.publisher,
               COUNT(b.book_id)
               FROM publishers AS p
               JOIN BOOKS AS b ON p.publisher_id=b.publisher_id
               WHERE b.num_pages > 50
               GROUP BY p.publisher_id
               ORDER BY count(b.book_id) DESC
               LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query3), con = con)

,publisher,count
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги, это максимальное количество.

**4. Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;**

In [16]:
query4 = ''' SELECT a.author,
            AVG(r.rating) AS rating_avg
            FROM books AS b FULL JOIN ratings AS r ON b.book_id = r.book_id
            FULL JOIN authors AS a ON b.author_id = a.author_id
            WHERE b.book_id IN (
                                SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(rating_id) >= 50
                                )
            GROUP BY a.author
            ORDER BY rating_avg DESC;
         '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query4), con = con)       

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


У J.K. Rowling/Mary GrandPré самый высокий рейтинг - 4.29.

**5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [17]:
query5 = '''WITH A AS
                    (SELECT username,
                     COUNT(review_id)
                     FROM reviews
                     WHERE username in
                         (SELECT username
                          FROM ratings
                          GROUP BY username
                          HAVING count(rating_id)>48)
                    GROUP BY username)
                SELECT round (SUM (COUNT) / COUNT(COUNT), 2) AS AVG
                FROM A'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query5), con = con)   

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.0

## Вывод

В ходе выполнения проектной работы был проведен анализ данных сервиса книг. А так же выполены поставленные задачи, по которым были сделаны следующие выводы:

1. После 1 января 2000 года вышло 819 книг;
2. Не все книги оценены, есть книги без оценок. Больше всего оценок у Twilight (Twilight #1).;
3. Издательство Penguin Books выпустило 42 книги, это максимальное количество;
4. У J.K. Rowling/Mary GrandPré самый высокий рейтинг - 4.29;
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.0.